<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
import matplotlib.pyplot as plt

# Custom dataset for time series
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        return (self.data[idx:idx+self.seq_length], self.data[idx+self.seq_length])

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=1):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Generate synthetic time series data
np.random.seed(0)
time_series_data = np.sin(np.linspace(0, 100, 1000))

seq_length = 10
dataset = TimeSeriesDataset(torch.tensor(time_series_data, dtype=torch.float32).unsqueeze(-1), seq_length)

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model, loss, and optimizer
model = LSTM()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training loop with validation
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

# Testing with new data
model.eval()
test_input = torch.tensor(time_series_data[-seq_length:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
with torch.no_grad():
    prediction = model(test_input)
print(f"Predicted next value: {prediction.item():.4f}")

# Plotting predictions
def plot_predictions(model, data, seq_length, num_predictions=50):
    model.eval()
    predictions = []
    for i in range(num_predictions):
        test_input = torch.tensor(data[-seq_length:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
        with torch.no_grad():
            prediction = model(test_input).item()
        predictions.append(prediction)
        data = np.append(data, prediction)

    plt.plot(np.linspace(0, 100, 1000 + num_predictions), np.sin(np.linspace(0, 100, 1000 + num_predictions)), label='Actual')
    plt.plot(np.linspace(100, 100 + num_predictions, num_predictions), predictions, label='Predicted')
    plt.legend()
    plt.show()

# Plot the predictions
plot_predictions(model, time_series_data, seq_length)